In [1]:
import pybel
import os
import logging
import nx2d3
import py2neo
import json
import time

time.asctime()

'Wed Oct  5 22:00:35 2016'

In [2]:
base = os.environ['PYBEL_BASE']
path = os.path.join(base, 'tests', 'bel', 'test_bel_1.bel')

In [3]:
g = pybel.from_bel(path)

In [4]:
nx2d3.embed_networkx(g, height=300, width=600)

<IPython.core.display.Javascript object>

In [5]:
print_edges = True
if print_edges:
    for u,v,data in g.edges(data=True):
        print(u, v, json.dumps(data, indent=2))

('Gene', 'ALPHABET', 'E') ('RNA', 'ALPHABET', 'E') {
  "relation": "transcribedTo"
}
('Gene', 'ALPHABET', 'C') ('RNA', 'ALPHABET', 'C') {
  "relation": "transcribedTo"
}
('Gene', 'ALPHABET', 'D') ('RNA', 'ALPHABET', 'D') {
  "relation": "transcribedTo"
}
('Gene', 'ALPHABET', 'B') ('RNA', 'ALPHABET', 'B') {
  "relation": "transcribedTo"
}
('Protein', 'ALPHABET', 'D') ('Protein', 'ALPHABET', 'E') {
  "TESTAN1": "2",
  "relation": "association",
  "citation_reference": "123456",
  "object": {
    "params": {
      "fromLoc": {
        "name": "intracellular",
        "namespace": "GOCC"
      },
      "toLoc": {
        "name": "cell surface",
        "namespace": "GOCC"
      }
    },
    "modification": "CellSurfaceExpression"
  },
  "citation_type": "Pubmed",
  "citation_name": "That other article from last week",
  "subject": {
    "params": {
      "molecularActivity": "KinaseActivity"
    },
    "modification": "Activity"
  },
  "Evidence": "Evidence 3"
}
('RNA', 'ALPHABET', 'C') ('

In [6]:
neo_path = os.environ['NEO_PATH']
neo = py2neo.Graph(neo_path)
neo.data('match (n) detach delete n')

[]

In [7]:
g.to_neo4j(neo)

{'relation': 'transcribedTo'}
{'relation': 'transcribedTo'}
{'relation': 'transcribedTo'}
{'relation': 'transcribedTo'}
{'TESTAN1': '2', 'relation': 'association', 'subject_modification': 'Activity', 'object_params_fromLoc_namespace': 'GOCC', 'object_params_toLoc_value': 'cell surface', 'citation_reference': '123456', 'object_modification': 'CellSurfaceExpression', 'object_params_fromLoc_value': 'intracellular', 'citation_name': 'That other article from last week', 'citation_type': 'Pubmed', 'Evidence': 'Evidence 3', 'object_params_toLoc_namespace': 'GOCC'}
{'relation': 'translatedTo'}
{'relation': 'translatedTo'}
{'relation': 'directlyDecreases', 'TESTAN2': '3', 'citation_type': 'Pubmed', 'citation_name': 'That one article from last week', 'citation_reference': '123455', 'Evidence': 'Evidence 2'}
{'relation': 'decreases', 'TESTAN2': '3', 'citation_type': 'Pubmed', 'citation_name': 'That one article from last week', 'citation_reference': '123455', 'Evidence': 'Evidence 2'}
{'relation':

In [8]:
neo.data('match (n) return n')

[{'n': (`0`:Protein {name:"0",namespace:"ALPHABET",value:"E"})},
 {'n': (`13`:RNA {name:"13",namespace:"ALPHABET",value:"B"})},
 {'n': (`1`:Protein {name:"1",namespace:"ALPHABET",value:"C"})},
 {'n': (`7`:RNA {name:"7",namespace:"ALPHABET",value:"C"})},
 {'n': (`8`:RNA {name:"8",namespace:"ALPHABET",value:"E"})},
 {'n': (`6`:Protein {name:"6",namespace:"ALPHABET",value:"D"})},
 {'n': (`3`:Gene {name:"3",namespace:"ALPHABET",value:"C"})},
 {'n': (`2`:Gene {name:"2",namespace:"ALPHABET",value:"E"})},
 {'n': (`9`:Protein {name:"9",namespace:"ALPHABET",value:"B"})},
 {'n': (`14`:Protein {name:"14",namespace:"ALPHABET",value:"A"})},
 {'n': (`4`:Gene {name:"4",namespace:"ALPHABET",value:"D"})},
 {'n': (`10`:RNA {name:"10",namespace:"ALPHABET",value:"D"})},
 {'n': (`11`:RNA {name:"11",namespace:"ALPHABET",value:"A"})},
 {'n': (`5`:Gene {name:"5",namespace:"ALPHABET",value:"B"})},
 {'n': (`12`:Gene {name:"12",namespace:"ALPHABET",value:"A"})}]

In [9]:
r = neo.run('match (n)-[k]-(m) return n.namespace, n.value, k.relation, m.namespace, m.value')
r.dump()

 n.namespace  n.value  k.relation         m.namespace  m.value 
---------------------------------------------------------------
 ALPHABET     E        translatedTo       ALPHABET     E       
 ALPHABET     E        association        ALPHABET     D       
 ALPHABET     B        translatedTo       ALPHABET     B       
 ALPHABET     B        transcribedTo      ALPHABET     B       
 ALPHABET     C        decreases          ALPHABET     B       
 ALPHABET     C        translatedTo       ALPHABET     C       
 ALPHABET     C        transcribedTo      ALPHABET     C       
 ALPHABET     C        translatedTo       ALPHABET     C       
 ALPHABET     E        transcribedTo      ALPHABET     E       
 ALPHABET     E        translatedTo       ALPHABET     E       
 ALPHABET     D        directlyDecreases  ALPHABET     B       
 ALPHABET     D        translatedTo       ALPHABET     D       
 ALPHABET     D        association        ALPHABET     E       
 ALPHABET     C        transcribedTo    